In [3]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [4]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import os 
import numpy as np
import pandas as pd

import glob
import random

from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AdaptiveAvgPool2d
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader

In [5]:
#Hyper parameters
train_datapath='/content/drive/MyDrive/1creditproject/train.npy'
train_labelpath='/content/drive/MyDrive/1creditproject/label_train.npy'
 #Give path related to wav files
modelpath='/content/drive/MyDrive/1creditproject/models/' #path to store models
classes=10
epochs=50
learning_rate=0.001
batch=16





In [6]:
train_data=np.load(train_datapath)
train_label=np.load(train_labelpath)

In [7]:
print(torch.cuda.device_count())

1


In [9]:
#set device
# Device will determine whether to run the training on GPU or CPU.
cuda0 = torch.device('cuda:0')
device = cuda0

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [10]:
#CNN model
class CNN(nn.Module):
    def __init__(self,classes):
        super(CNN,self).__init__()
        self.relu = nn.ReLU()
        self.max1=MaxPool2d(kernel_size=(5,5)) 
        self.max2=MaxPool2d(kernel_size=(4,90)) 
        self.drop=Dropout(0.3)
        self.conv1=Conv2d(in_channels=1,out_channels=32,kernel_size=(7,7),padding=(2,2))
        self.b1=BatchNorm2d(32)
        self.conv2=Conv2d(in_channels=32,out_channels=64,kernel_size=(7,7),padding=(2,2))
        self.b2=BatchNorm2d(64)
        self.gap = AdaptiveAvgPool2d((1,1))
        self.lin1=Linear(64,100)
        self.lin2=Linear(100,10)
    def forward(self,x):
       # print(x.shape)
        x=(self.relu(self.b1(self.conv1(x))))
        x=self.max1(x)
        x=self.drop(x)
        x=(self.relu(self.b2(self.conv2(x))))
        x=self.max2(x)
        x=self.drop(x)
        x=self.gap(x)
#         print(x.shape)
        x = x.view(x.shape[0],-1)
#         print(x.shape)
        x=self.lin1(x)
        
        x=self.lin2(x)
        
        return x
        
           
        




In [11]:
#Initialize network
model = CNN(classes)
model.to(device)
model.double()

#Loss and optimizer
 #Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = Adam(model.parameters(),lr=learning_rate)

In [12]:
#Load data

train_loader=[]
for i in range(len(train_data)):
  x,y=train_data[i],train_label[i]
  x=x.reshape(40,501,1)
  x=torch.from_numpy(x)
  x=x.permute(2,0,1)
  x.numpy()
  y1=np.array([y]) 
  ydata1 = torch.from_numpy(y1)
#   print(x.shape)
#   print(y.shape)
  train_loader.append((x,ydata1))
  

data=DataLoader(train_loader, batch_size=batch, shuffle=True)
# print(data[1].shape)





In [ ]:
'''def extract(scale_file):
    # load audio files with librosa
    scale, sr = librosa.load(scale_file)
    mel_spectrogram = librosa.feature.melspectrogram(scale, sr=sr, n_fft=882, hop_length=441, n_mels=40)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    clas=scale_file[28:31]
    clas1=scale_file[34:39]
    Ydata1 = []
    
    if   clas == 'air':
        Y = 0
    elif clas == 'bus':
        Y = 1
    elif clas == 'met':
        if clas1=='stati':
            Y=3
        else:
            Y = 2
    elif clas == 'par':
        Y = 4
    elif clas == 'pub':
        Y = 5
    elif clas == 'sho':
        Y = 6     
    elif clas == 'str':
        if clas1=='_pede':
            Y=7
        else:
            Y = 8

    elif clas == 'tra':
        Y = 9
    

    Y1=np.array([Y]) 
    Ydata1 = torch.from_numpy(Y1)  
    return log_mel_spectrogram,Ydata1'''

In [13]:
#Train network
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(epochs):
	#Load in the data in batches using the train_loader object
    for batchid,(images, labels) in enumerate(data):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        #print(labels)
        # Forward pass
        outputs = model(images)
        #print(labels.shape)
        labels = torch.squeeze(labels,1)
        loss = criterion(outputs, labels)
        #print(labels.shape)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))
    path = modelpath+str(epoch+1)+".pth"
    torch.save(model.state_dict(),path)

Epoch [1/50], Loss: 1.1499
Epoch [2/50], Loss: 1.4121
Epoch [3/50], Loss: 1.0320
Epoch [4/50], Loss: 1.3064
Epoch [5/50], Loss: 1.1917
Epoch [6/50], Loss: 0.5950
Epoch [7/50], Loss: 0.9389
Epoch [8/50], Loss: 1.3227
Epoch [9/50], Loss: 1.2639
Epoch [10/50], Loss: 0.6996
Epoch [11/50], Loss: 0.8295
Epoch [12/50], Loss: 0.9749
Epoch [13/50], Loss: 0.3087
Epoch [14/50], Loss: 0.6168
Epoch [15/50], Loss: 0.5214
Epoch [16/50], Loss: 1.1612
Epoch [17/50], Loss: 0.7176
Epoch [18/50], Loss: 0.7270
Epoch [19/50], Loss: 1.4157
Epoch [20/50], Loss: 1.0979
Epoch [21/50], Loss: 0.5703
Epoch [22/50], Loss: 0.2969
Epoch [23/50], Loss: 0.5516
Epoch [24/50], Loss: 1.0540
Epoch [25/50], Loss: 0.8647
Epoch [26/50], Loss: 0.3400
Epoch [27/50], Loss: 1.0248
Epoch [28/50], Loss: 0.5839
Epoch [29/50], Loss: 0.2595
Epoch [30/50], Loss: 0.7084
Epoch [31/50], Loss: 0.4320
Epoch [32/50], Loss: 1.5537
Epoch [33/50], Loss: 0.7682
Epoch [34/50], Loss: 0.9498
Epoch [35/50], Loss: 0.4669
Epoch [36/50], Loss: 0.4253
E